In [275]:
import pandas as pd

In [276]:
data_allocine = pd.read_csv("input_datasets/allocine.csv")
data_imdb = pd.read_csv("input_datasets/imdbmovies.csv")




In [277]:
# display(data_allocine.info())

In [278]:
# display(data_imdb.info())

formatage date :


In [279]:
mois_numeriques = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

# Fonction pour convertir le format de date pour allocine
def format_date_allocine(date_string):

    for m in mois_numeriques.keys():
        if m in date_string:
            splits = date_string.split()
            day = str(splits[0])
            month = str(mois_numeriques[splits[1]])
            year = str(splits[-1])
            date_format = day + "/" + month + "/" + year
            return date_format

    date_format = str(date_string.split(' au ')[0])
    return date_format

# Fonction pour convertir le format de date pour imdb
def convert_date_format(date_str):
    try:
        return pd.to_datetime(date_str).strftime('%d/%m/%Y')
    except:
        return None

# Convertir les dates du csv d'allocine
data_allocine['semaine_fr'] = data_allocine['semaine_fr'].apply(format_date_allocine)

# Appliquer la fonction pour convertir le format de date
data_imdb['date_sortie_y'] = data_imdb['date_sortie_y'].apply(convert_date_format)

# Supprime les données avec no date found / None
data_imdb = data_imdb[(data_imdb["date_sortie_y"] != 'No date found') & (data_imdb["date_sortie_y"].notna())]

# Afficher les premières lignes pour vérification
display(data_imdb["date_sortie_y"].info())

# Supprimer les données avec dates antérieurs à 2000

<class 'pandas.core.series.Series'>
Index: 2245 entries, 1 to 2499
Series name: date_sortie_y
Non-Null Count  Dtype 
--------------  ----- 
2245 non-null   object
dtypes: object(1)
memory usage: 35.1+ KB


None

Gestion du Budget:

In [280]:
data_imdb['budget'] = data_imdb['budget'].str.replace(' (estimated)', '')

data_imdb['budget'] = data_imdb['budget'].str.replace(',', '')
data_imdb['budget'] = data_imdb['budget'].str.replace(' ', '')

# Supprimer les lignes avec 'No budger' dans la colonne 'budget'
data_imdb = data_imdb[data_imdb['budget'] != 'No budget']

# Réinitialiser l'index
data_imdb.reset_index(drop=True, inplace=True)

# Afficher les premières lignes pour vérification
# display(data_imdb)


In [281]:
import re

taux_de_conversion = {
    '$': 1.0,
    '£': 1.38,  # Livre sterling (taux de change actuel)
    '€': 1.10,  # Euro (taux de change actuel)
    '₹': 0.014,  # Roupie indienne (taux de change actuel)
    'DEM': 0.62,  # Deutsche Mark (taux de change historique)
    '¥': 0.0093,  # Yen japonais (taux de change actuel)
    'FRF': 0.18,  # Franc français (taux de change historique)
    '₩': 0.00089,  # Won sud-coréen (taux de change actuel)
    'ATS': 0.090,  # Schilling autrichien (taux de change historique)
    'RUR': 0.013,  # Rouble russe (taux de change actuel)
    'DKK': 0.16,  # Couronne danoise (taux de change actuel)
    'CN¥': 0.15,  # Yuan chinois (taux de change actuel)
    'SEK': 0.12,  # Couronne suédoise (taux de change actuel)
    'ITL': 0.00061  # Lire italienne (taux de change historique)
}

# Boucler à travers les valeurs et extraire les symboles et les montants
for i, valeur in enumerate(data_imdb['budget']):
    match = re.match(r'([^$\d]+)(\d+)', valeur)
    if match:
        symbole = match.group(1).strip()
        montant = int(match.group(2))
        if symbole in taux_de_conversion:
            montant *= taux_de_conversion[symbole]
        data_imdb.at[i, 'budget'] = montant

data_imdb['budget'] = data_imdb['budget'].str.replace('$', '')


# Afficher les premières lignes pour vérification
# display(data_imdb)



In [282]:
display(data_allocine)

,actors,director,entrees_fr,entrees_usa,image_urls,nationalite,semaine_fr,semaine_usa,studio,timing,title,titre_original
0,"Maxime Gasteuil,Zabou Breitman,Romain Lancry","Edouard Pluvieux,Lionel Dutemple,Edouard Pluvieux",126678,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,france,06/03/2024,NaN,wild bunch distribution,0,14 jours pour aller mieux,pas de titre original
1,"Agnès Jaoui,William Lebghil,Salif Cissé","Julien Carpentier,Julien Carpentier,Benjamin G...",58613,NaN,https://fr.web.img4.acsta.net/c_310_420/pictur...,france,06/03/2024,NaN,kmbo,0,la vie de ma mère,pas de titre original
2,"Christian Friedel,Sandra Hüller,Johann Karthaus","Jonathan Glazer,Jonathan Glazer",239108,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,u.s.a.,31/01/2024,NaN,bac films,0,la zone d'intérêt,the zone of interest
3,"Audrey Lamy,Fabrice Eboué,Anouk Grinberg","Maxime Govare,Romain Choay,Maxime Govare,Romai...",136628,NaN,https://fr.web.img6.acsta.net/c_310_420/pictur...,france,13/03/2024,NaN,warner bros. france,0,heureux gagnants,pas de titre original
4,"Paola Cortellesi,Valerio Mastandrea,Romana Mag...","Paola Cortellesi,Paola Cortellesi,Furio Andreotti",145340,NaN,https://fr.web.img3.acsta.net/c_310_420/pictur...,italie,13/03/2024,NaN,universal pictures international france,0,il reste encore demain,c'è ancora domani
...,...,...,...,...,...,...,...,...,...,...,...,...
1721,"Joel Edgerton,Callum Turner,Peter Guinness","George Clooney,Mark L. Smith",8401000,NaN,https://fr.web.img6.acsta.net/c_310_420/img/99...,u.s.a.,29/12/2024,NaN,NaN,0,ils étaient un seul homme,the boys in the boat
1722,"Hilary Swank,Alan Ritchson,Amy Acker","Jon Gunn,Kelly Fremon Craig",6500000,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,23/02/2024,NaN,NaN,0,ordinary angels,pas de titre original
1723,"Ariana DeBose,Chris Messina,Pilou Asbæk","Gabriela Cowperthwaite,Nick Shafir",3064583,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,u.s.a.,19/01/2024,NaN,1 nomination,0,i.s.s.,pas de titre original
1724,"Chuku Modu,Kit Young,Safia Oakley-Green","Andrew Cumming,Ruth Greenberg",0,NaN,https://fr.web.img5.acsta.net/c_310_420/pictur...,grande-bretagne,09/02/2024,NaN,NaN,0,out of darkness,pas de titre original
